In [1]:
from jetcam.csi_camera import CSICamera

import numpy as np
import cv2
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget1 = ipywidgets.Image(format='jpeg')
image_widget2 = ipywidgets.Image(format='jpeg')
image_widget3 = ipywidgets.Image(format='jpeg')
image_widget4 = ipywidgets.Image(format='jpeg')
image_widget5 = ipywidgets.Image(format='jpeg')
camera = CSICamera(width=300, height=300)
print("camera_connected")

camera_connected


In [2]:
def get_contours(image, threshold=0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    return contours

In [3]:
def update_image(widget, frame):
    widget.value = bgr8_to_jpeg(frame)

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 0.5
color = (255, 0, 0)
thickness = 2


def create_object_marker(image, area, x, y, w, h):
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.putText(image, f"bottom: {h + y}", (x + 5, y + 15), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(image, f"center: {x + int(w / 2)}", (x + 5, y + 30), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(image, f"area: {area}", (x + 5, y + 45), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.circle(image, (x + int(w / 2), h + y), 5, (0, 0, 255), -1)
    return image

In [5]:
tl = (85, 50)
bl = (0, 300)
tr = (215, 50)
br = (300, 300)

pts1 = np.float32([tl, bl, tr, br])
pts2 = np.float32([[0, 0], [0, 300], [300, 0], [300, 300]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)


def get_objects(image):
    image = cv2.warpPerspective(image, matrix, (300, 300))
    mask = background_filter(image)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    contours = get_contours(masked_image)

    objects = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 6000:
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 3)
            x, y, w, h = cv2.boundingRect(contour)
            objects.append([x, y, w, h])
            image = create_object_marker(image, area, x, y, w, h)

    return objects, image

In [6]:
def get_object_distance(object):
    return object[1] + object[3]

In [ ]:
# done = False
# i = 0
# display(image_widget1)
# display(image_widget2)
# display(image_widget3)
# display(image_widget4)
# display(image_widget5)
# while not done:
#     frame = camera.read()
#     update_image(image_widget1, frame)
#     frame = warp_perspective(frame)
#     update_image(image_widget2, frame)
#     mask = background_filter(frame)
#     update_image(image_widget3, mask)
#     masked_image = cv2.bitwise_and(frame, frame, mask=mask)
#     update_image(image_widget4, masked_image)
#
#     contours = get_contours(masked_image)
#
#     objects = []
#
#     for contour in contours:
#         area = cv2.contourArea(contour)
#         if area > 6000:
#             cv2.drawContours(frame, [contour], -1, (0, 255, 0), 3)
#             x, y, w, h = cv2.boundingRect(contour)
#             objects.append([x, y, w, h])
#             frame = create_object_marker(frame, area, x, y, w, h)
#     update_image(image_widget5, frame)
#
#     i += 1
#     if i > 1000:
#         done = True


In [7]:
def callback(change):
    frame = change['new']
    update_image(image_widget1, frame)
    frame = warp_perspective(frame)
    update_image(image_widget2, frame)
    mask = background_filter(frame)
    update_image(image_widget3, mask)
    masked_image = cv2.bitwise_and(frame, frame, mask=mask)
    update_image(image_widget4, masked_image)

    contours = get_contours(masked_image)

    objects = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area >200:
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 3)
            x, y, w, h = cv2.boundingRect(contour)
            objects.append([x, y, w, h])
            frame = create_object_marker(frame, area, x, y, w, h)
    update_image(image_widget5, frame)

In [8]:
kernel = np.ones((3, 3), np.uint8)


def background_filter(image):
    image = cv2.blur(image, (10, 10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # lower = np.array([0, 0, 50])
    # upper = np.array([255, 75, 180])
    # lower = np.array([0, 0, 0])
    # upper = np.array([200, 95, 200])
    lower = np.array([0, 0, 50])
    upper = np.array([255, 95, 255])
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, kernel, iterations=10)
    mask = cv2.dilate(mask, kernel, iterations=10)
    mask = cv2.bitwise_not(mask)
    return mask

In [9]:
tl = (85, 60)
bl = (0, 300)
tr = (215, 60)
br = (300, 300)

pts1 = np.float32([tl, bl, tr, br])
pts2 = np.float32([[0, 0], [0, 300], [300, 0], [300, 300]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)


def warp_perspective(image):
    image = cv2.warpPerspective(image, matrix, (300, 300))
    return image

In [10]:
display(image_widget1)
display(image_widget2)
display(image_widget3)
display(image_widget4)
display(image_widget5)
camera.running = True
camera.observe(callback, names='value')

Image(value=b'', format='jpeg')

Image(value=b'', format='jpeg')

Image(value=b'', format='jpeg')

Image(value=b'', format='jpeg')

Image(value=b'', format='jpeg')

In [ ]:
# camera.unobserve(callback, names='value')